In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install ultralytics

In [ ]:
#training 1
import os
import cv2
import numpy as np
from ultralytics import YOLO
import torch
import albumentations as A
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Function to apply bilateral filtering and CLAHE
def preprocess_image(img):

    # Apply bilateral filtering for noise reduction while preserving edges
    img_filtered = cv2.bilateralFilter(img, 9, 75, 75)

    # Convert to LAB color space for CLAHE
    lab = cv2.cvtColor(img_filtered, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to the L-channel (lightness)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)

    # Merge CLAHE enhanced L-channel back with A and B channels
    lab_clahe = cv2.merge((l_clahe, a, b))

    # Convert back to BGR color space
    img_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

    return img_clahe

# Fine-tuning MobileNetV2 for classification (drowsy vs awake)
def load_mobilenetv2():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Adding a custom classification head (binary classification: drowsy or awake)
    predictions = Dense(1, activation='sigmoid')(x)

    # Creating the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to classify eye/mouth state using MobileNetV2
def classify_region(region, model):
    if region.size == 0 or region.shape[0] == 0 or region.shape[1] == 0:  # Check if the region is empty
        return 'Unknown'

    region_resized = cv2.resize(region, (224, 224))  # Resize to 224x224
    region_array = img_to_array(region_resized)
    region_preprocessed = preprocess_input(np.expand_dims(region_array, axis=0))
    prediction = model.predict(region_preprocessed)

    if prediction > 0.5:
        return 'Drowsy'
    else:
        return 'Awake'

# Train the YOLO model with the correct parameters
def train_model():
    model = YOLO('yolov8s.pt')
    results = model.train(
        data= '/content/drive/MyDrive/finalyolo/datasetv8_dev.yml',
        epochs=4,
        workers=8,
        lr0=0.001,  # Set a lower learning rate
        batch=64,     # Reduce batch size to prevent memory issues
        amp=True,   
        save=True,
        save_period=1,
        cache=True
    )
    print("Training complete.")
    return model

# Augmentation pipeline using Albumentations for random resizing
augmentation_pipeline = A.Compose([
    A.RandomResizedCrop(width=224, height=224, scale=(0.8, 1.0), ratio=(0.9, 1.1), p=1.0)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Load YOLOv8 and MobileNetV2 models
model = train_model()
mobilenet_model = load_mobilenetv2()

# Inference and visualization with MobileNetV2 and preprocessing
def inference_and_visualize(model, mobilenet_model, images_folder):
    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)
        img = cv2.imread(image_path)

        img_preprocessed = preprocess_image(img)
        h, w, _ = img_preprocessed.shape

        # Perform inference with YOLOv8
        results = model(img_preprocessed)

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy().astype(np.int64)
            for box in boxes:
                x1, y1, x2, y2 = [int(coord) for coord in box]

                x1 = max(0, min(x1, w))
                y1 = max(0, min(y1, h))
                x2 = max(0, min(x2, w))
                y2 = max(0, min(y2, h))

                region = img_preprocessed[y1:y2, x1:x2]

                # Apply augmentation before classification
                transformed = augmentation_pipeline(image=region, bboxes=[[x1/w, y1/h, (x2-x1)/w, (y2-y1)/h]], class_labels=[0])
                transformed_image = transformed['image']
                state = classify_region(transformed_image, mobilenet_model)

                if state != 'Unknown':  # Only annotate valid regions
                    cv2.rectangle(img_preprocessed, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(img_preprocessed, state, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow('Drowsiness Detection', img_preprocessed)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Perform inference and visualization on validation images
inference_and_visualize(model, mobilenet_model, "/content/drive/MyDrive/finalyolo/final_dataset/images/val_resized")


KeyboardInterrupt: 

In [ ]:
# Reinstall PyTorch
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Reinstall Ultralytics (YOLOv8)
!pip install ultralytics --upgrade


Found existing installation: torch 2.4.1+cu121
Uninstalling torch-2.4.1+cu121:
  Successfully uninstalled torch-2.4.1+cu121
Found existing installation: torchvision 0.19.1+cu121
Uninstalling torchvision-0.19.1+cu121:
  Successfully uninstalled torchvision-0.19.1+cu121
Found existing installation: torchaudio 2.4.1+cu121
Uninstalling torchaudio-2.4.1+cu121:
  Successfully uninstalled torchaudio-2.4.1+cu121
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 122.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
#FINAL TRAINING 1
import os
import cv2
import numpy as np
from ultralytics import YOLO
import torch
import albumentations as A
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Function to apply bilateral filtering and CLAHE
def preprocess_image(img):
    # Apply bilateral filtering for noise reduction while preserving edges
    img_filtered = cv2.bilateralFilter(img, 9, 75, 75)

    # Convert to LAB color space for CLAHE
    lab = cv2.cvtColor(img_filtered, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to the L-channel (lightness)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)

    # Merge CLAHE enhanced L-channel back with A and B channels
    lab_clahe = cv2.merge((l_clahe, a, b))

    # Convert back to BGR color space
    img_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

    return img_clahe

# Fine-tuning MobileNetV2 for classification (drowsy vs awake)
def load_mobilenetv2():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Adding a custom classification head (binary classification: drowsy or awake)
    predictions = Dense(1, activation='sigmoid')(x)

    # Creating the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to classify eye/mouth state using MobileNetV2
def classify_region(region, model):
    if region.size == 0 or region.shape[0] == 0 or region.shape[1] == 0:  # Check if the region is empty
        return 'Unknown'

    region_resized = cv2.resize(region, (224, 224))  # Resize to 224x224
    region_array = img_to_array(region_resized)
    region_preprocessed = preprocess_input(np.expand_dims(region_array, axis=0))
    prediction = model.predict(region_preprocessed)

    if prediction > 0.5:
        return 'Drowsy'
    else:
        return 'Awake'

# Train the YOLO model with the correct parameters, enabling mosaic augmentation
def train_model():
    model = YOLO('yolov8s.pt')
    results = model.train(
        data='/content/drive/MyDrive/finalyolo/datasetv8_dev.yml',
        epochs=15,
        workers=8,
        lr0=0.001,  # Set a lower learning rate
        batch=64,   # Reduce batch size to prevent memory issues
        amp=True,   # Enable Automatic Mixed Precision
        save=True,
        save_period=1,
        cache=True,
        mosaic=1.0  # Enable mosaic augmentation (default 1.0 is full strength)
    )
    print("Training complete.")
    return model

# Augmentation pipeline using Albumentations with additional techniques
augmentation_pipeline = A.Compose([
    A.RandomResizedCrop(width=224, height=224, scale=(0.8, 1.0), ratio=(0.9, 1.1), p=1.0),  # Random crop
    A.HorizontalFlip(p=0.5),  # Flip horizontally with 50% probability
    A.RandomBrightnessContrast(p=0.5),  # Adjust brightness and contrast with 50% probability
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),  # Small shifts, scaling, and rotations
    A.CLAHE(p=0.1),  # CLAHE for low-light conditions
    A.Blur(p=0.05)   # Apply a slight blur
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Load YOLOv8 and MobileNetV2 models
model = train_model()
mobilenet_model = load_mobilenetv2()

# Inference and visualization with MobileNetV2 and preprocessing
def inference_and_visualize(model, mobilenet_model, images_folder):
    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)
        img = cv2.imread(image_path)

        img_preprocessed = preprocess_image(img)
        h, w, _ = img_preprocessed.shape

        # Perform inference with YOLOv8
        results = model(img_preprocessed)

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy().astype(np.int64)
            for box in boxes:
                x1, y1, x2, y2 = [int(coord) for coord in box]

                x1 = max(0, min(x1, w))
                y1 = max(0, min(y1, h))
                x2 = max(0, min(x2, w))
                y2 = max(0, min(y2, h))

                region = img_preprocessed[y1:y2, x1:x2]

                # Apply augmentation before classification
                transformed = augmentation_pipeline(image=region, bboxes=[[x1/w, y1/h, (x2-x1)/w, (y2-y1)/h]], class_labels=[0])
                transformed_image = transformed['image']
                state = classify_region(transformed_image, mobilenet_model)

                if state != 'Unknown':  # Only annotate valid regions
                    cv2.rectangle(img_preprocessed, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(img_preprocessed, state, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow('Drowsiness Detection', img_preprocessed)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Perform inference and visualization on validation images
inference_and_visualize(model, mobilenet_model, "/content/drive/MyDrive/finalyolo/final_dataset/images/val_resized")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


100%|██████████| 21.5M/21.5M [00:00<00:00, 184MB/s]


Ultralytics YOLOv8.2.101 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/finalyolo/datasetv8_dev.yml, epochs=15, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=1, cache=True, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sho

100%|██████████| 755k/755k [00:00<00:00, 15.0MB/s]


...

                   all       6929       6929      0.997      0.996      0.994      0.936



15 epochs completed in 0.683 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.101 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:34<00:00,  1.59it/s]


                   all       6929       6929      0.997      0.996      0.994      0.936
                 awake       3317       3317      0.998      0.996      0.994      0.962
                drowsy       3612       3612      0.996      0.997      0.994       0.91
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train
Training complete.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

0: 640x640 1 drowsy, 8.3ms
Speed: 2.2ms preprocess, 8.3ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)


ValueError: In YOLO format all coordinates must be float and in range (0, 1], got [[          0           0     0.99107           1           0]]

In [ ]:
#FINAL TRAINING 2
import os
import cv2
import numpy as np
from ultralytics import YOLO
import torch
import albumentations as A
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Function to apply bilateral filtering and CLAHE
def preprocess_image(img):
    # Apply bilateral filtering for noise reduction while preserving edges
    img_filtered = cv2.bilateralFilter(img, 9, 75, 75)

    # Convert to LAB color space for CLAHE
    lab = cv2.cvtColor(img_filtered, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to the L-channel (lightness)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)

    # Merge CLAHE enhanced L-channel back with A and B channels
    lab_clahe = cv2.merge((l_clahe, a, b))

    # Convert back to BGR color space
    img_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

    return img_clahe

# Fine-tuning MobileNetV2 for classification (drowsy vs awake)
def load_mobilenetv2():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Adding a custom classification head (binary classification: drowsy or awake)
    predictions = Dense(1, activation='sigmoid')(x)

    # Creating the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to classify eye/mouth state using MobileNetV2
def classify_region(region, model):
    if region.size == 0 or region.shape[0] == 0 or region.shape[1] == 0:  # Check if the region is empty
        return 'Unknown'

    region_resized = cv2.resize(region, (224, 224))  # Resize to 224x224
    region_array = img_to_array(region_resized)
    region_preprocessed = preprocess_input(np.expand_dims(region_array, axis=0))
    prediction = model.predict(region_preprocessed)

    if prediction > 0.5:
        return 'Drowsy'
    else:
        return 'Awake'

# Function to continue YOLO model training from saved model for an additional 15 epochs
def continue_training():
    model = YOLO('/content/drive/MyDrive/finalyolo/train_results/weights/last.pt')  # Load the last checkpoint
    results = model.train(
        data='/content/drive/MyDrive/finalyolo/datasetv8_dev.yml',
        epochs=15,  # Train for 15 more epochs
        workers=8,
        lr0=0.001,  # Set a lower learning rate
        batch=64,   # Reduce batch size to prevent memory issues
        amp=True,   # Enable Automatic Mixed Precision
        save=True,
        save_period=1,
        cache=True,
        mosaic=1.0  # Enable mosaic augmentation (default 1.0 is full strength)
    )
    print("Training continued and complete.")
    return model

# Augmentation pipeline using Albumentations with additional techniques
augmentation_pipeline = A.Compose([
    A.RandomResizedCrop(width=224, height=224, scale=(0.8, 1.0), ratio=(0.9, 1.1), p=1.0),  # Random crop
    A.HorizontalFlip(p=0.5),  # Flip horizontally with 50% probability
    A.RandomBrightnessContrast(p=0.5),  # Adjust brightness and contrast with 50% probability
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),  # Small shifts, scaling, and rotations
    A.CLAHE(p=0.1),  # CLAHE for low-light conditions
    A.Blur(p=0.05)   # Apply a slight blur
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Continue the training from the saved model checkpoint
model = continue_training()
mobilenet_model = load_mobilenetv2()

# Inference and visualization with MobileNetV2 and preprocessing
def inference_and_visualize(model, mobilenet_model, images_folder):
    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)
        img = cv2.imread(image_path)

        img_preprocessed = preprocess_image(img)
        h, w, _ = img_preprocessed.shape

        # Perform inference with YOLOv8
        results = model(img_preprocessed)

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy().astype(np.int64)
            for box in boxes:
                x1, y1, x2, y2 = [int(coord) for coord in box]

                x1 = max(0, min(x1, w))
                y1 = max(0, min(y1, h))
                x2 = max(0, min(x2, w))
                y2 = max(0, min(y2, h))

                region = img_preprocessed[y1:y2, x1:x2]

                # Apply augmentation before classification
                transformed = augmentation_pipeline(image=region, bboxes=[[x1/w, y1/h, (x2-x1)/w, (y2-y1)/h]], class_labels=[0])
                transformed_image = transformed['image']
                state = classify_region(transformed_image, mobilenet_model)

                if state != 'Unknown':  # Only annotate valid regions
                    cv2.rectangle(img_preprocessed, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(img_preprocessed, state, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow('Drowsiness Detection', img_preprocessed)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Perform inference and visualization on validation images
inference_and_visualize(model, mobilenet_model, "/content/drive/MyDrive/finalyolo/final_dataset/images/val_resized")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Ultralytics YOLOv8.2.101 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/finalyolo/train_results/weights/last.pt, data=/content/drive/MyDrive/finalyolo/datasetv8_dev.yml, epochs=15, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=1, cache=True, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, s

100%|██████████| 755k/755k [00:00<00:00, 119MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              


...

                   all       6929       6929      0.997      0.998      0.994      0.942



15 epochs completed in 0.668 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.101 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:30<00:00,  1.82it/s]


                   all       6929       6929      0.997      0.998      0.994      0.942
                 awake       3317       3317      0.996      0.998      0.994      0.967
                drowsy       3612       3612      0.997      0.997      0.994      0.918
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train
Training continued and complete.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

0: 640x640 1 drowsy, 8.5ms
Speed: 2.2ms preprocess, 8.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


ValueError: In YOLO format all coordinates must be float and in range (0, 1], got [[          0           0     0.99554           1           0]]

In [ ]:
import shutil

# Specify the source file (zipped results) and the destination path in Google Drive
source_file = 'train_results.zip'
destination = '/content/drive/MyDrive/train_results.zip'

# Copy the file to Google Drive
shutil.copy(source_file, destination)

print("File copied to Google Drive successfully.")



FileNotFoundError: [Errno 2] No such file or directory: 'train_results.zip'

In [ ]:
#FINAL TRAINING 3
import os
import cv2
import numpy as np
from ultralytics import YOLO
import torch
import albumentations as A
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Function to apply bilateral filtering and CLAHE
def preprocess_image(img):
    # Apply bilateral filtering for noise reduction while preserving edges
    img_filtered = cv2.bilateralFilter(img, 9, 75, 75)

    # Convert to LAB color space for CLAHE
    lab = cv2.cvtColor(img_filtered, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to the L-channel (lightness)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)

    # Merge CLAHE enhanced L-channel back with A and B channels
    lab_clahe = cv2.merge((l_clahe, a, b))

    # Convert back to BGR color space
    img_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

    return img_clahe

# Fine-tuning MobileNetV2 for classification (drowsy vs awake)
def load_mobilenetv2():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Adding a custom classification head (binary classification: drowsy or awake)
    predictions = Dense(1, activation='sigmoid')(x)

    # Creating the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to classify eye/mouth state using MobileNetV2
def classify_region(region, model):
    if region.size == 0 or region.shape[0] == 0 or region.shape[1] == 0:  # Check if the region is empty
        return 'Unknown'

    region_resized = cv2.resize(region, (224, 224))  # Resize to 224x224
    region_array = img_to_array(region_resized)
    region_preprocessed = preprocess_input(np.expand_dims(region_array, axis=0))
    prediction = model.predict(region_preprocessed)

    if prediction > 0.5:
        return 'Drowsy'
    else:
        return 'Awake'

# Function to continue YOLO model training from saved model for an additional 15 epochs
def continue_training():
    model = YOLO('/content/drive/MyDrive/finalyolo/runs/train_results (1)/weights/best.pt') # Load the last checkpoint
    results = model.train(
        data='/content/drive/MyDrive/finalyolo/datasetv8_dev.yml',
        epochs=15,  # Train for 15 more epochs
        workers=8,
        lr0=0.001,  # Set a lower learning rate
        batch=64,   # Reduce batch size to prevent memory issues
        amp=True,   # Enable Automatic Mixed Precision
        save=True,
        save_period=1,  # Save every epoch
        cache=True,
        mosaic=1.0  # Enable mosaic augmentation (default 1.0 is full strength)
    )

    # Manual saving of model after each epoch if needed
    for epoch in range(15):
        # Assume every epoch is processed here, you can save after each
        model.save(f'/content/drive/MyDrive/finalyolo/train_results/weights/epoch_{epoch}.pt')

    print("Training continued and complete.")
    return model


# Augmentation pipeline using Albumentations with additional techniques
augmentation_pipeline = A.Compose([
    A.RandomResizedCrop(width=224, height=224, scale=(0.8, 1.0), ratio=(0.9, 1.1), p=1.0),  # Random crop
    A.HorizontalFlip(p=0.5),  # Flip horizontally with 50% probability
    A.RandomBrightnessContrast(p=0.5),  # Adjust brightness and contrast with 50% probability
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),  # Small shifts, scaling, and rotations
    A.CLAHE(p=0.1),  # CLAHE for low-light conditions
    A.Blur(p=0.05)   # Apply a slight blur
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Continue the training from the saved model checkpoint
model = continue_training()
mobilenet_model = load_mobilenetv2()

# Inference and visualization with MobileNetV2 and preprocessing
def inference_and_visualize(model, mobilenet_model, images_folder):
    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)
        img = cv2.imread(image_path)

        img_preprocessed = preprocess_image(img)
        h, w, _ = img_preprocessed.shape

        # Perform inference with YOLOv8
        results = model(img_preprocessed)

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy().astype(np.int64)
            for box in boxes:
                x1, y1, x2, y2 = [int(coord) for coord in box]

                x1 = max(0, min(x1, w))
                y1 = max(0, min(y1, h))
                x2 = max(0, min(x2, w))
                y2 = max(0, min(y2, h))

                region = img_preprocessed[y1:y2, x1:x2]

                # Apply augmentation before classification
                transformed = augmentation_pipeline(image=region, bboxes=[[x1/w, y1/h, (x2-x1)/w, (y2-y1)/h]], class_labels=[0])
                transformed_image = transformed['image']
                state = classify_region(transformed_image, mobilenet_model)

                if state != 'Unknown':  # Only annotate valid regions
                    cv2.rectangle(img_preprocessed, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(img_preprocessed, state, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow('Drowsiness Detection', img_preprocessed)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Perform inference and visualization on validation images
inference_and_visualize(model, mobilenet_model, "/content/drive/MyDrive/finalyolo/final_dataset/images/val_resized")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Ultralytics YOLOv8.2.102 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/finalyolo/runs/train_results (1)/weights/best.pt, data=/content/drive/MyDrive/finalyolo/datasetv8_dev.yml, epochs=15, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=1, cache=True, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames

100%|██████████| 755k/755k [00:00<00:00, 144MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

...

                   all       6929       6929      0.997      0.997      0.994      0.946



15 epochs completed in 0.672 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.102 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:29<00:00,  1.85it/s]


                   all       6929       6929      0.998      0.997      0.994      0.946
                 awake       3317       3317      0.998      0.996      0.994      0.971
                drowsy       3612       3612      0.997      0.998      0.993       0.92
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train
Training continued and complete.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

0: 640x640 1 drowsy, 7.9ms
Speed: 2.3ms preprocess, 7.9ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


ValueError: In YOLO format all coordinates must be float and in range (0, 1], got [[          0           0     0.99554     0.99554           0]]

In [ ]:
import shutil
import os

# Path to the directory containing the training results
source_dir = 'runs/detect/train'  # Make sure this is the correct path to your training results

# Zip the results directory
shutil.make_archive('train_results', 'zip', source_dir)

# Specify the source file (zipped results) and the destination path in Google Drive
source_file = 'train_results.zip'
destination = '/content/drive/MyDrive/train_results.zip'

# Copy the zipped file to Google Drive
shutil.copy(source_file, destination)

print("Zipped results copied to Google Drive successfully.")


Zipped results copied to Google Drive successfully.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file generated by YOLOv8 training
results_path = '/path_to_yolo_training_folder/results.csv'
df = pd.read_csv(results_path)

# Plot training and validation losses over epochs
plt.figure(figsize=(10, 5))

# Plot box loss
plt.subplot(1, 2, 1)
plt.plot(df['epoch'], df['train/box_loss'], label='train/box_loss')
plt.plot(df['epoch'], df['val/box_loss'], label='val/box_loss')
plt.xlabel('Epoch')
plt.ylabel('Box Loss')
plt.title('Box Loss over Epochs')
plt.legend()

# Plot class loss
plt.subplot(1, 2, 2)
plt.plot(df['epoch'], df['train/cls_loss'], label='train/cls_loss')
plt.plot(df['epoch'], df['val/cls_loss'], label='val/cls_loss')
plt.xlabel('Epoch')
plt.ylabel('Class Loss')
plt.title('Class Loss over Epochs')
plt.legend()

plt.tight_layout()
plt.show()
